# Train on Remote Virtual Machines

Train MLflow Projects on remote DSVMs (Data Science Virtual Machines).

## Table of Contents
1. Prerequisites
    - 1.1 Initialize Tracking Store and Experiment
    - 1.2 Create and Attach DSVM 
    - 1.3 Configure the Backend Configuration object
    - 1.4 Modify your Environment Specification
3. Submit Run


# Prerequisites 
Ensure you have done the following before running this notebook,
- Connected to an AML Workspace
- Have an existing remote DSVM in that Workspace
- Have an MLproject file with an environment specification

In [ ]:
# Prereq Checks

# Workspace check
from azureml.core import Workspace

workspace = Workspace.from_config()
print(workspace.name, workspace.resource_group, workspace.location, workspace.subscription_id, sep = '\n')

# Existing DSVM check
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

dsvm_name = "dsvm"
try:
    cpu_cluster = ComputeTarget(workspace = workspace, name = dsvm_name)
    print("Found existing cluster, yay!")
except ComputeTargetException:
    print("This compute target is not associated with your workspace!")


### Create and Attach a DSVM as a Compute Target
You can spin up a DSVM in two ways:

1. Create a DSVM in your resource group using the following command
```
az vm create --resource-group <resource_group_name> --name <some_vm_name>
--image microsoft-dsvm:linux-data-science-vm-ubuntu:linuxdsvmubuntu:latest
--admin-username <username> --admin-password <password>
--generate-ssh-keys --authentication-type password
```
2. Go to the [Azure Portal](https://ms.portal.azure.com/#home) and in the search bar, type "Data Science Virtual Machine". On the right under "Marketplace", there should be an option to select "Data Science Virtual Machine - Ubuntu 18.04". Select 'Create' and add the required information. Set the region to be in Central US EUAP. 

## Initialize Tracking Store and Experiment

### Set Tracking URI
 
Set the MLflow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLflow APIs will go to Azure ML services and will be tracked under your Workspace.

In [ ]:
from azureml import core
from azureml.core import Workspace

import mlflow

workspace = Workspace.from_config()  
mlflow.set_tracking_uri(workspace.get_mlflow_tracking_uri())

### Create Experiment

Create an Mlflow Experiment to organize your runs. It can be set either by passing the name as a **parameter** in the mlflow.projects.run call or by the following,

In [ ]:
experiment_name = "mlflow-example"
mlflow.set_experiment(experiment_name)

## Create the Backend Configuration Object

The backend configuration object will store necesary information for the integration such as the compute target and whether to use your local managed environment or a system managed environment. 

The integration will accept "COMPUTE" and "USE_CONDA" as parameters where "COMPUTE" is set to the name of your remote compute cluster and "USE_CONDA" which creates a new environment for the project from the environment configuration file. If "COMPUTE" is present in the object, the project will be automatically submitted to the remote compute and ignore "USE_CONDA". Mlflow accepts a dictionary object or a JSON file.

In [ ]:
# dictionary
backend_config = {"COMPUTE": "dsvm", "USE_CONDA": False}


## Modify your Environment specification

Add the azureml-mlflow package as a pip dependency to your environment configuration file. The project can run without this addition, but key artifacts and metrics will not be logged to your Workspace. Adding it to to the file will look like this,

```
name: mlflow-example
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - python=3.6
  - scikit-learn=0.19.1
  - pip
  - pip:
    - mlflow
    - azureml-mlflow
```

# Submit Run 



In [ ]:
remote_mlflow_run = mlflow.projects.run(uri=".", 
                                    parameters={"alpha":0.3},
                                    backend = "azureml",
                                    backend_config = backend_config)